In [1]:
from astropy.table import Table, vstack, MaskedColumn
from astropy.io import fits
from astropy.coordinates import SkyCoord
import numpy as np
import bisect as bs
from astropy import units as u
import matplotlib.pyplot as plt

# Set up functions to load the data

In [2]:
# Takes in a list of masks and returns an astropy table containing all of the primus outputs
def load_primus_data(masklist):
    primus_table = Table()
    for mask in masklist:
        catalog ='primus_catalogs/catalog_edsc0'+str(mask)+'.fits'

        hdu = fits.getdata(catalog, ext=0, header=True)
        ralist = hdu[0][0]
        declist = hdu[0][1]
        zlist = hdu[0][2]
        zerr = hdu[0][3]
        zconf = hdu[0][4]
        zgood = hdu[0][5]
        ra_spec = hdu[0][6]
        dec_spec = hdu[0][7]

        ttable = Table([ralist, declist, zlist, zerr, zconf, zgood, ra_spec, dec_spec],
                       names=('ra', 'dec', 'zLDP', 'zLDPerr', 'Q', 'zLDP_good', 'ra_slit', 'dec_slit'))
        primus_table = vstack([primus_table, ttable])

    photpos = SkyCoord(ra=primus_table['ra']*u.degree, dec=primus_table['dec']*u.degree)
    slitpos = SkyCoord(ra=primus_table['ra_slit']*u.degree, dec=primus_table['dec_slit']*u.degree)
    primus_table['slit_distance'] = photpos.separation(slitpos).arcsecond

    primus_table.sort('ra')
    primus_table['uid'] = np.arange(0, len(primus_table['ra']), 1)
    print("Done importing PRIMUS data")
    return primus_table

In [3]:
def load_zspecs():
    names, zspecs, zspecQs, spectype = np.genfromtxt('zspecs/all.zspec.dat', dtype=str, usecols=[0,1,2,5], unpack=True)
    # Parse names into RA and DEC
    zras, zdecs = [], []
    for name in names:
        tra = name[6:13]
        tdec = name[13:]
        tempra = "%ih%im%i.%is" % (int(tra[0:2]), int(tra[2:4]), int(tra[4:6]), int(tra[6:]))
        tempdec = "%sd%sm%s.%ss" % (tdec[0:3], tdec[3:5], tdec[5:7], tdec[7:])
        c = SkyCoord(tempra, tempdec, frame='icrs')

        zras.append(c.ra.deg)
        zdecs.append(c.dec.deg)
    print("Done importing FORS2 data")
    return Table([zras, zdecs, zspecs.astype(float), zspecQs.astype(int), spectype],
                 names=('ra', 'dec', 'zSpec', 'zSpec_quality', 'spec_type'))

# Load the PRIMUS outputs

In [4]:
# Make list of masks for PRIMUS catalogs
masklist = np.arange(245, 281, 1).tolist()
masklist.remove(249)
masklist.remove(258)
masklist.remove(267)

In [5]:
primus_table = load_primus_data(masklist)
primus_table['zSpec'] = -1.
primus_table['zSpec'].mask = True
primus_table['zSpec_quality'] = -1
primus_table['zSpec_quality'].mask = True
primus_table['spec_type'] = ''
primus_table['spec_type'].mask = True

Done importing PRIMUS data


# Load the FORS2 zspec data

In [6]:
zspec_table = load_zspecs()

Done importing FORS2 data


In [7]:
print(zspec_table.colnames)

['ra', 'dec', 'zSpec', 'zSpec_quality', 'spec_type']


# Match the two catalogs

In [8]:
primus_coords = SkyCoord(ra=primus_table['ra']*u.degree, dec=primus_table['dec']*u.degree)
zspec_coords = SkyCoord(ra=zspec_table['ra']*u.degree, dec=zspec_table['dec']*u.degree)
pidx, pd2d, pd3d = primus_coords.match_to_catalog_sky(zspec_coords)

In [9]:
primus_table['zSpec'] = zspec_table['zSpec'][pidx]
primus_table['zSpec_quality'] = zspec_table['zSpec_quality'][pidx]
primus_table['spec_type'] = zspec_table['spec_type'][pidx]
primus_table['zspec_dist_asec'] = pd2d.arcsecond

# Remask any entries with zspec_dist_asec > 1 as

In [10]:
maskidx = np.where(primus_table['zspec_dist_asec'] > 1.0)
primus_table['zSpec'][maskidx] = -1
primus_table['zSpec_quality'][maskidx] = -1
primus_table['spec_type'][maskidx] = ''

In [11]:
print(len(primus_table[np.where(primus_table['zSpec'] > 0.0)]))

539


# Output the matched table

In [12]:
primus_table.write('personal_catalogs/primus_fors2.csv', format='csv', overwrite=True)

In [13]:
primus_table.colnames

['ra',
 'dec',
 'zLDP',
 'zLDPerr',
 'Q',
 'zLDP_good',
 'ra_slit',
 'dec_slit',
 'slit_distance',
 'uid',
 'zSpec',
 'zSpec_quality',
 'spec_type',
 'zspec_dist_asec']